In [1]:
import pandas as pd
import os
import sys
import boto3

sys.path.append(os.path.expanduser('~'))
from scripts.utils.write_metadata import (
    make_metadata_files, 
    append_metadata
)

In [2]:
df = pd.read_csv('Full Data Pipeline Notes - 1_ Pull.csv')

# drop empty columns
df = df.loc[:, df.columns.notna()]
# do some housekeeping
df = df.rename(
    columns={
        'Ideas for AWS naming' : 'Variable',
        'Data' : 'Source Dataset Name',
        'Date of First Pull ' : 'Date Pulled',
        'Data Puller' : 'Data Pulled By',
    }
)
df = df.drop(columns=['Pulled Format','Notes','Link',df.columns[0]])
pd.set_option('display.max_rows', 10)

df


,Source Dataset Name,Domain,Indicator,Metric,Date Pulled,Data Pulled By,Date of Second Pull (if applicable),Second Pull Justification,Variable,Data Download URL,Unnamed: 14
0,Broadband Internet Availability,Built Environment,Communication,% of area with braodband speeds of 25 mpbs dow...,11/3/2023,Jesse Espinoza,NaN,NaN,built_cpuc_internet,https://www.cpuc.ca.gov/industries-and-topics/...,NaN
1,Radio transmission towers,Built Environment,Communication,# of towers,11/4/2023,Jesse Espinoza,NaN,NaN,built_hiflld_radio_towers,https://hifld-geoplatform.opendata.arcgis.com/...,NaN
2,Cellular towers,Built Environment,Communication,# of towers,11/5/2023,Jesse Espinoza,NaN,NaN,built_hifld_cellular_towers,https://hifld-geoplatform.opendata.arcgis.com/...,NaN
3,Microwave service towers,Built Environment,Communication,# of towers,11/6/2023,Jesse Espinoza,NaN,NaN,built_hifld_microwave_towers,https://hifld-geoplatform.opendata.arcgis.com/...,NaN
4,Paging transmission towers,Built Environment,Communication,# of towers,11/7/2023,Jesse Espinoza,NaN,NaN,built_hifld_paging_towers,https://hifld-geoplatform.opendata.arcgis.com/...,NaN
...,...,...,...,...,...,...,...,...,...,...,...
107,Demographic,Society & Economy,Vulnerable populations,% of population aged 65 or older,10/27/2023,Jesse Espinoza,12/19/2023,Re-pulled for new 5-year ACS 2022 data,society_acs_demographic,https://data.census.gov/,NaN
108,Demographic,Society & Economy,Vulnerable populations,% of population under 5 years old,10/27/2023,Jesse Espinoza,12/19/2023,Re-pulled for new 5-year ACS 2022 data,society_acs_demographic,https://data.census.gov/,NaN
109,Demographic,Society & Economy,Vulnerable populations,Race: American Indian and Alaska Native,10/27/2023,Jesse Espinoza,12/19/2023,Re-pulled for new 5-year ACS 2022 data,society_acs_demographic,https://data.census.gov/,NaN
110,Health insurance,Society & Economy,Vulnerable populations,% of population without health insurance,10/27/2023,Jesse Espinoza,12/20/2023,Re-pulled for new 5-year ACS 2022 data,socieaty_acs_health_insurance,https://data.census.gov/,NaN


In [3]:
make_metadata_files(df)

### Manually placed all .txt files into a folder called 'pull_metadata_files'

In [9]:
s3_client = boto3.client('s3')  
bucket_name = 'ca-climate-index'  
directory = '3_fair_data/metadata/1_pull'
folder_path = 'pull_metadata_files'

In [10]:
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)

    with open(file_path, 'rb') as data:
        s3_key = f"{directory}/{filename}"
        s3_client.upload_fileobj(data, bucket_name, s3_key)
        print(f"Uploaded {filename} to S3 at {s3_key}")

print("Upload complete.")

Uploaded built_acs_housing_quality_metadata.txt to S3 at 3_fair_data/metadata/1_pull/built_acs_housing_quality_metadata.txt
Uploaded built_acs_housing_vacancy_metadata.txt to S3 at 3_fair_data/metadata/1_pull/built_acs_housing_vacancy_metadata.txt
Uploaded built_acs_mobile_homes_metadata.txt to S3 at 3_fair_data/metadata/1_pull/built_acs_mobile_homes_metadata.txt
Uploaded built_caltrans_airports_metadata.txt to S3 at 3_fair_data/metadata/1_pull/built_caltrans_airports_metadata.txt
Uploaded built_caltrans_bridges_metadata.txt to S3 at 3_fair_data/metadata/1_pull/built_caltrans_bridges_metadata.txt
Uploaded built_caltrans_highways_metadata.txt to S3 at 3_fair_data/metadata/1_pull/built_caltrans_highways_metadata.txt
Uploaded built_caltrans_rails_metadata.txt to S3 at 3_fair_data/metadata/1_pull/built_caltrans_rails_metadata.txt
Uploaded built_caltrans_road_bottlenecks_metadata.txt to S3 at 3_fair_data/metadata/1_pull/built_caltrans_road_bottlenecks_metadata.txt
Uploaded built_cec_power_p